In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime
import pickle
from dotenv import dotenv_values

config = dotenv_values("../.env")
auth = {
    'Authorization': f"Bearer {config['API_TOKEN']}",
    'User-Agent': 'wzzzz/1.0 (oformleno@gmail.com)',
        }

In [7]:
def get_vacancies(page=0):
    return requests.get(
                        'https://api.hh.ru/vacancies'
                        '?area=113&area=40&area=28&area=1001'
                        '&text=python'
                        '&search_field=name&search_field=description'
                        '&employment=full&employment=part'
                        '&schedule=remote'
                        '&professional_role=96'
                        '&industry=7&industry=8&industry=9&industry=11'
                        '&industry=39.448&industry=39.444&industry=39.445'
                        '&industry=39.446&industry=41.525'
                        '&industry=43&industry=44.391&industry=44.396'
                        '&industry=44.392&industry=44.393&industry=49.404'
                        '&industry=49.411&industry=49.412&industry=389'
                        '&date_from=2023-01-01'
                        f'&page={page}'
                        '&per_page=100',
                        # headers=auth
                            )

In [8]:
vacancies = get_vacancies()

In [9]:
def get_df(items):
    df = pd.DataFrame.from_records(items)
    df.drop(columns=[
                    'has_test',
                    'premium',
                    'address',
                    'department',
                    'response_letter_required',
                    'sort_point_distance',
                    'response_url',
                    'relations',
                    'schedule',
                    'working_days',
                    'working_time_intervals',
                    'working_time_modes',
                    'accept_temporary',
                    'apply_alternate_url',
                    'insider_interview',
                    'adv_response_url',
                    'contacts',    
                ], inplace=True)
    df['area'] = df['area'].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
    df['type'] = df['type'].apply(lambda x: x.get('name'))
    df['employer'] = df['employer'].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['snippet'], axis=1), df['snippet'].apply(pd.Series)], axis=1)
    return df

In [11]:
df = pd.DataFrame()
for page in range(vacancies.json()['pages']):
    v = get_vacancies(page=page)
    d = get_df(v.json()['items'])
    df = df.append(d, ignore_index=True)

/tmp/ipykernel_103202/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_103202/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_103202/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_103202/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future 

In [12]:
with open("../data/indexes", "rb") as fp:   # Unpickling
    indexes = pickle.load(fp)
indexes = set(indexes)
print(len(indexes))

1214


In [13]:
deep_vac = []
for i in df.itertuples():
    if i.url and i.id not in indexes:
        r = requests.get(
            i.url,
            # headers=auth
            )
        deep_vac.append(r.json())

In [16]:
deep_df = pd.DataFrame.from_records(deep_vac)

In [17]:
def get_deep_df(df):
    df = df.drop(columns=[
        'premium',
        'billing_type',
        'relations',
        'accept_handicapped',
        'insider_interview',
        'response_letter_required',
        'type',
        'address',
        'allow_messages',
        'schedule',
        'department',
        'contacts',
        'contacts',
        'branded_description',
        'vacancy_constructor_template',
        'accept_kids',
        'archived',
        'response_url',
        'code',
        'quick_responses_allowed',
        'driver_license_types',
        'accept_incomplete_resumes',
        'initial_created_at',
        'negotiations_url',
        'suitable_resumes_url',
        'has_test',
        'test',
        'working_days',
        'working_time_intervals',
        'working_time_modes',
        'accept_temporary',
        'hidden',
                ])
    
    for name in ['area', 'employer', 'experience', 'employment']:
        df[name] = df[name].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
    
    def skills_to_list(x):
        if x:
            if isinstance(x[0], dict):
                items = [i['name'] for i in x]
                return items
            else:
                return x

    for name in ['key_skills', 'specializations', 'professional_roles']:
        df[name] = df[name].apply(skills_to_list)
        
    df['languages'] = df['languages'].apply(lambda x: x if x else None)
    
    def get_text(x):
        if x:
            soup = bs(x, features="html.parser")
            text = soup.get_text()
            return text
        
    df['description'] = df['description'].apply(get_text)
    
    return df

In [18]:
deep_df1 = get_deep_df(deep_df)

/tmp/ipykernel_103202/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_103202/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_103202/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_103202/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a fut

In [20]:
stop_words = [
 'Angular',
 'C#',
 'C++',
 'Hotcore',
 'Laravel',
 'Lua',
 'NixOS',
 'Oracle',
 'React',
 'Ruby',
 '.NET',
 '1С',
 '.NET',
 'C/Go',
 'DilOS',
 'Golang',
 'Frontend',
 'Full-Stack',
 'Fullstack',
 'Java',
 'Javascript',
 'Next.js)',
 'Node.js',
 'PHP',
 'Perl',
 'Rails',
 'Scala',
 'Unity',
]
drop = []
for i in deep_df1.itertuples():
    if i.name:
        for j in stop_words:
            if j.lower() in i.name.lower():
                drop.append(i.id)
                break

In [21]:
deep_df3 = deep_df1.drop(deep_df1[deep_df1['id'].isin(drop)].index.to_list())

In [22]:
deep_df3.sort_values(by=['employer'], inplace=True)

In [23]:
deep_df3.drop(
    ['employment', 'specializations', 'published_at', 
     'created_at', 'apply_alternate_url', 'gross', 'professional_roles'], 
    axis=1, 
    inplace=True
    )

In [24]:
deep_df3.drop(deep_df3[deep_df3['languages'].notnull()].index, inplace=True)
deep_df3.drop(['languages'], axis=1, inplace=True)

In [25]:
deep_df3.sort_values(by=['employer'], inplace=True)

In [28]:
k_skills = set()
for i in deep_df3['key_skills']:
    if i:
        k_skills.update(list(i))

In [29]:
stop_words = [
    'Английский язык', '1С программирование', '.NET', '.NET Core', '.NET',
    'Adobe Photoshop', 'C#', 'C++', 'C/C++', 'Java', 'JavaScript', 'Lua', 
    'Node.js', 'PHP', 'Perl', 'React', 'React.js', 'SCALA', 'Vue.js', 'С#',
    'С++',
]
for i in deep_df3.itertuples():
    if i.key_skills:
        for j in stop_words:
            if j in i.key_skills:
                drop.append(i.id)
                break
deep_df4 = deep_df3.drop(deep_df3[deep_df3['id'].isin(drop)].index.to_list())

In [30]:
deep_df4.drop_duplicates(subset=['description'], inplace=True)
deep_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 46 to 27
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             100 non-null    object 
 1   name           100 non-null    object 
 2   area           100 non-null    object 
 3   experience     100 non-null    object 
 4   description    100 non-null    object 
 5   key_skills     63 non-null     object 
 6   employer       100 non-null    object 
 7   alternate_url  100 non-null    object 
 8   from           9 non-null      float64
 9   to             21 non-null     float64
 10  currency       23 non-null     object 
dtypes: float64(2), object(9)
memory usage: 9.4+ KB


In [31]:
deep_df4.tail()

,id,name,area,experience,description,key_skills,employer,alternate_url,from,to,currency
71,78142179,"Код-ревьюер курса ""Мидл python-разработчик"" на...",Москва,От 3 до 6 лет,Яндекс Практикум — это сервис онлайн-образован...,"[Python, Django Framework, Redis, Elasticsearc...",Яндекс Практикум,https://hh.ru/vacancy/78142179,NaN,NaN,NaN
88,78142115,"Наставник курса ""Мидл python-разработчик"" на b...",Москва,От 3 до 6 лет,Яндекс Практикум — это сервис онлайн-образован...,"[Python, PostgreSQL, Django, Elasticsearch, fa...",Яндекс Практикум,https://hh.ru/vacancy/78142115,NaN,NaN,NaN
45,78153426,Код-ревьюер на курс “Асинхронное программирова...,Москва,От 3 до 6 лет,"Яндекс Практикум — сервис онлайн-образования, ...","[Python, asyncio, aiohttp, Linux, GIL, FastAPI...",Яндекс Практикум,https://hh.ru/vacancy/78153426,NaN,NaN,NaN
43,78153449,Наставник на курс “Асинхронное программировани...,Москва,От 3 до 6 лет,"Яндекс Практикум — сервис онлайн-образования, ...","[Python, asyncio, Грамотная речь, aiohttp, GIL...",Яндекс Практикум,https://hh.ru/vacancy/78153449,NaN,NaN,NaN
27,78075638,Инженер данных (PostgreSQL),Москва,От 3 до 6 лет,Наша команда каждый день работает над повышени...,"[Python, PostgreSQL, аналитика]",билайн,https://hh.ru/vacancy/78075638,NaN,NaN,NaN


In [32]:
dt = datetime.datetime.now().strftime("%m-%d-%Y(%H-%M)")
deep_df4.to_csv(f'../data/result_deep_sorted_{dt}.csv', index=False)

In [33]:
indexes.update(deep_df4.id)
with open("../data/indexes", "wb") as fp:   #Pickling
    pickle.dump(indexes, fp)
print(len(indexes))

1314
